In [12]:
# 1. load data
import os
import pandas as pd
import numpy as np

from src.data_loader import load_and_prepare_projects
from src.llm_client import get_embeddings

path_16 = 'data/wrc16_projects.csv'
path_17 = 'data/wrc17_projects.csv'

df, df16_shuffled = load_and_prepare_projects(path_16,path_17, city = 'Wroclaw')
df

,project_id,project_name,description,category,cost,district,votes,rank,year
0,710,Drzewa dla Wrocławia - nasadzenia w całym mieś...,Uzasadnienie\nZnikające drzewa z krajobrazu mi...,greenery/recreation,1000000,"Kuźniki, Nowy Dwór, Muchobór Mały,Grabiszyn-Gr...",13938,1,2016
1,15,Zielona rowerowo-piesza obwodnica Wrocławia; E...,Uzasadnienie,walking/cycling infrastructure,1000000,"Grabiszyn-Grabiszynek, Krzyki-Partynice, Oporó...",12348,2,2016
2,764,Oświetlenie Parku Grabiszyńskiego (Alei Romera...,Uzasadnienie\nTaki mamy klimat... że przez pół...,other,1000000,"Grabiszyn-Grabiszynek, Krzyki-Partynice, Oporó...",7291,3,2016
3,685,Plac Zabaw dla Starszaków w parku Grabiszyński...,Uzasadnienie\nPlac Zabaw dla Starszaków w park...,playgrounds,560000,"Huby, Gaj, Tarnogaj,Nadodrze, Ołbin, Stare Mia...",6663,4,2016
4,379,"Parking przy ""Dobrzyńskiej"" - ułatwienie dojaz...",Uzasadnienie\nProjekt polega na utworzeniu w c...,roads,750000,"Gajowice, Powstańców Śląskich, Borek",6383,5,2016
...,...,...,...,...,...,...,...,...,...
97,422,"Akcja Plac - Gry uliczne ""Oswajamy beton""",Uzasadnienie\nProjekt zakłada lokowanie w prze...,greenery/recreation,150000,Ołbin,236,46,2017
98,406,Budowa wrocławskiej wypożyczalni rowerów integ...,Uzasadnienie\nProjekt przeznaczony jest zarówn...,walking/cycling infrastructure,600000,Tarnogaj,178,47,2017
99,629,Wrocław na dotknięcie ręki – tylfograficzne ma...,"Uzasadnienie\nRynek i wszystkie miejsca, takie...",other,130000,"Stare Miasto, Przedmieście Świdnickie,Przedmie...",156,48,2017
100,720,1997,"Uzasadnienie\nIdeą projektu ""1997"" było upamię...",other,50000,Ołtaszyn,118,49,2017


In [ ]:
api_key = os.getenv('OPENAI_API_KEY')

df['text'] = df['project_name'] + ": " + df['description']
df['embedding'] = df['text'].apply(lambda x: get_embeddings(x, api_key))
df['embedding'] = df['embedding'].apply(lambda x: np.array(x).tolist())

In [ ]:
results = df.filter(['project_id', 'project_name', 'year', 'embedding'])
results.to_csv('output/proj16_and_17_pl_embeddings_openai-3-large.csv', sep=";", index=False)

In [ ]:
class2022= pd.read_csv('data/proj2022_topics_gpt4-turbo.csv', sep=";")
class2024= pd.read_csv('data/proj2024_topics_gpt4-turbo.csv', sep=";")

pclass = pd.concat([class2022,class2024])
pclass

t1 = pd.merge(left=embedding_df, right=pclass, on='project_id', how = "left")
t1

In [ ]:

import ast

embedding_df2022 = pd.read_csv('data/proj2022_embeddings_openai-3-large.csv')
embedding_df2022['embedding'] = embedding_df2022['embedding'].apply(ast.literal_eval)

embedding_df2024 = pd.read_csv('data/proj2024_embeddings_openai-3-large.csv')
embedding_df2024['embedding'] = embedding_df2024['embedding'].apply(ast.literal_eval)

embedding_df = pd.concat([embedding_df2022, embedding_df2024]).reset_index(drop=True)
embedding_df['embedding'] = embedding_df['embedding'].apply(np.array)

embedding_df